## 빅데이터 실습

### 다나와 무선청소기 상품 분석

#### 데이터 수집

##### 셀레니움 사용
- 무선청소기 검색에 사용할 URL
    https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&checkedInfo=N&volumeType=allvs&page=2&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A
- URL 최소화 하면
    https://search.danawa.com/dsearch.php?query=무선청소기&page=2&limit=120&sort=saveDESC&list=list
    한페이지 다 로드될 때까지 약 5초정도 소요

In [70]:
# 필수 라이브러리 사용 등록
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse    #url encode 위함
from tqdm import tqdm, tqdm_notebook
import time

In [5]:
# 웹드라이버로 크롬 오픈
driver = webdriver.Chrome()
url = r'https://search.danawa.com/dsearch.php?query=무선청소기&page=2&limit=120&sort=saveDESC&list=list'
driver.get(url)
time.sleep(5.0)

In [10]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [14]:
prodItems = soup.select('ul.product_list>li.prod_item')

In [25]:
# id = "productItem19965230" 식으로 전개되므로 productItem이 아니면 패스
# 제품명 가져오기, 에드스마트 영역: 예외발생
prodItems[0].select('p.prod_name>a')[0].text.strip()   

'에어메이드 스마트포스 AMC-3500'

In [33]:
# 제품 사양
prodItems[0].select('div.spec_list')[0].text.strip().replace("\t", "")

'핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 370W / 2022년형 / [구성] 충전 / 스탠드거치 / 브러쉬: 바닥 / 침구 / 솔형 / 틈새 / [배터리] 사용시간: 45분(최대) / 충전시간: 5시간 / 분리형(1개) / 리튬이온 / 2500mAh / [청소] LED라이트 / UV살균 / [부가] 디스플레이표시 / 헤파필터 / 5단계여과 / 무게: 2.5kg / 크기(가로x세로x깊이): 255x1230x215mm'

In [38]:
# 제품 가격 가져오기
# 교재: 1위 데이터 가져옴
# 우리: input 중 두번째 값이 최소값 담고 있는 것으로 확인
prodItems[0].select('input')[1].get('value')

'197000'

In [39]:
#제품 인트로
prodItems[1].select('p.intro_text')[0].text

IndexError: list index out of range

##### 1번 페이지에서 상품 정보 추출

In [44]:
prodData = []
loop = 0
for prodItem in prodItems:
    try:
        prodName = prodItem.select('p.prod_name > a')[0].text.strip()   #상품명 가져오기
        specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '')   #상품 스펙목록 가져오기
        price = prodItem.select('input')[1].get('value')    #최저값 가져오기
        loop += 1
        prodData.append([prodName, specList, price])

    except:
        pass


In [46]:
# 결과 확인
prodData

[['에어메이드 스마트포스 AMC-3500',
  '핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 370W / 2022년형 / [구성] 충전 / 스탠드거치 / 브러쉬: 바닥 / 침구 / 솔형 / 틈새 / [배터리] 사용시간: 45분(최대) / 충전시간: 5시간 / 분리형(1개) / 리튬이온 / 2500mAh / [청소] LED라이트 / UV살균 / [부가] 디스플레이표시 / 헤파필터 / 5단계여과 / 무게: 2.5kg / 크기(가로x세로x깊이): 255x1230x215mm',
  '197000'],
 ['일렉트로룩스 Well Q8 WQ81-3OMBF',
  '핸디스틱청소기 / 무선 / 흡입형 / 2021년형 / [구성] 충전 / 스탠드거치 / 브러쉬: 바닥 / 솔형 / 틈새 / [배터리] 사용시간: 53분(최대) / 충전시간: 4시간30분 / 내장형 / [청소] 싸이클론흡입 / LED라이트 / 먼지통: 0.3L / [부가] 셀프스탠딩 / 헤파필터 / 5단계여과 / 색상: 마호가니브론즈 / 무게: 2.8kg / 크기(가로x세로x깊이): 258ㅌ1105ㅌ140mm',
  '329000'],
 ['삼성전자 비스포크 제트 VS20A957E3',
  '핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 210W / 2021년형 / [구성] 거치대: 청정스테이션 / 먼지비움 / 충전 / 스탠드거치 / 먼지비움시간: 20초 / 브러쉬: 바닥 / 물걸레: 일반 / 침구 / 솔형 / 틈새 / 연장관 / 먼지봉투: 1.2L / [배터리] 사용시간: 1시간(최대) / 충전시간: 3시간30분 / 분리형(2개) / 리튬이온 / 2200+2200mAh / 25.2V / [청소] 싸이클론흡입 / 자동물공급 / 물분사 / 디지털인버터모터 / 먼지통: 0.5L / [부가] 대화형알림창 / 디스플레이표시 / 배터리잔량표시 / 워셔블헤파필터 / 5단계여과 / 색상: 미스티화이트 / 무게: 2.5kg / 크기(가로x세로x깊이): 250x930x202mm',
  '592730'],
 ['쿠

##### 여러 페이지 크롤링 방법

- URL
    - https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A

In [67]:
# 검색어, 페이지를 변경하면서 URL 생성해주는 함수
def getSearchPageUrl(keyword, page):
    ecKeyword = parse.quote(keyword)
    url = f'https://search.danawa.com/dsearch.php?query={ecKeyword}&originalQuery={ecKeyword}&previousKeyword={ecKeyword}&checkedInfo=N&volumeType=allvs&' + \
        f'page={page}&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&' + \
        f'recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&' + \
        f'quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
    return url

In [68]:
# 검색어에 ' '(스페이스)가 있으면 URL에서 링크 클릭이 안되는 문제 발생 -> urlEncode: parse.quote()
print(getSearchPageUrl('갤럭시 s24', 1))

https://search.danawa.com/dsearch.php?query=%EA%B0%A4%EB%9F%AD%EC%8B%9C%20s24&originalQuery=%EA%B0%A4%EB%9F%AD%EC%8B%9C%20s24&previousKeyword=%EA%B0%A4%EB%9F%AD%EC%8B%9C%20s24&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A


In [72]:
total_page = 10
for page in tqdm(range(1, total_page + 1)):
    # print('페이지 처리')    #tqdm 진행바가 여러번 출력됨(보기 안좋음)
    time.sleep(1)

100%|██████████| 10/10 [00:10<00:00,  1.00s/it]


In [75]:
# 상품 정보 추출 함수
def getProdItems(prodItems):
    prodData = []

    for prodItem in prodItems:
        try:
            prodName = prodItem.select('p.prod_name > a')[0].text.strip()   #상품명 가져오기
            specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '')   #상품 스펙목록 가져오기
            price = prodItem.select('input')[1].get('value')    #최저값 가져오기
            prodData.append([prodName, specList, price])

        except:
            pass
    
    return prodData

In [82]:
# 여러 페이지 검색 후 크롤링하는 작업
driver = webdriver.Chrome()
# 암묵적으로 웹 자원 로드를 위해 3초정도 대기
driver.implicitly_wait(3.0)  #implicitly: 암묵적으로

keyword = '무선청소기'
totalPage = 20
prodDataTotal = []  #최종적으로 저장할 리스트

for page in tqdm(range(1, totalPage+1)):
    #검색 페이지 이동
    url = getSearchPageUrl(keyword, page)
    driver.get(url)
    #페이지 로딩이 완료될 때까지 5초간 대기
    time.sleep(5)
    #현재 페이지 HTML 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    #상품정보 추출
    prodItems = soup.select('ul.product_list > li.prod_item')
    prodItemList = getProdItems(prodItems)

    #추출된 정보를  prodDataTotal에 추가
    prodDataTotal += prodItemList

100%|██████████| 20/20 [03:55<00:00, 11.76s/it]


In [84]:
len(prodDataTotal)

2119

In [85]:
dfProdData = pd.DataFrame(prodDataTotal)

In [87]:
dfProdData.to_excel('./data/다나와_무선청소기_결과.xlsx', index=False)